In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://pics.awwmemes.com/okay-we-are-go-for-launch-blastoff-hah-ha-ha-ha-do9mo-comics-56166934.png)awwmemes.com

In [ ]:
df = pd.read_csv('../input/spaceship-titanic/train.csv')
pd.set_option('display.max_columns', None)
test = pd.read_csv('../input/spaceship-titanic/test.csv')
sub = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')

In [ ]:
from sklearn.feature_selection import mutual_info_regression

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
#Code by Parul Pandey  https://www.kaggle.com/parulpandey/a-guide-to-handling-missing-values-in-python


from sklearn.impute import SimpleImputer
df_most_frequent = df.copy()
#setting strategy to 'mean' to impute by the mean
mean_imputer = SimpleImputer(strategy='most_frequent')# strategy can also be mean or median 
df_most_frequent.iloc[:,:] = mean_imputer.fit_transform(df_most_frequent)

In [ ]:
df_most_frequent.isnull().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder

#fill in mean for floats
for c in df.columns:
    if df[c].dtype=='float16' or  df[c].dtype=='float32' or  df[c].dtype=='float64':
        df[c].fillna(df[c].mean())

#fill in -999 for categoricals
df = df.fillna(-999)
# Label Encoding
for f in df.columns:
    if df[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(df[f].values))
        df[f] = lbl.transform(list(df[f].values))
        
print('Labelling done.')

In [ ]:
df['Transported'] = df['Transported'].astype(int)

In [ ]:
df.head()

In [ ]:
# Utility functions from Tutorial
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    color = np.array(["C0"] * scores.shape[0])
    # Color red for probes
    idx = [i for i, col in enumerate(scores.index)
           if col.startswith("PROBE")]
    color[idx] = "C3"
    # Create plot
    plt.barh(width, scores, color=color)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [ ]:
#X=Value and col= Variable are Not columns of the Dataset. Therefore, Do Not change them!

features = ["VIP", "Cabin", "VRDeck"]
sns.relplot(
    x="value", y="Transported", col="variable", data=df.melt(id_vars="Transported", value_vars=features), facet_kws=dict(sharex=False),
);

In [ ]:
X = df.copy()
y = X.pop('Transported')

mi_scores = make_mi_scores(X, y)

#ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.

#This notebook is an exercise in the Feature Engineering course by Ryan Holbrook.

https://www.kaggle.com/ryanholbrook/clustering-with-k-means

In [ ]:
#Code by Ryan H. https://www.kaggle.com/ryanholbrook/clustering-with-k-means

def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",#metric is Auc in this Titanic Spaceship
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score

In [ ]:
#Code by Ryan H. https://www.kaggle.com/ryanholbrook/clustering-with-k-means

X = df.copy()
y = X.pop("Transported")


# YOUR CODE HERE: Define a list of the features to be used for the clustering
features = [
    "Age",
    "VIP",
    "Destination",
    "CryoSleep",
    "Cabin",
]


# Standardize
X_scaled = X.loc[:, features]
X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)


# YOUR CODE HERE: Fit the KMeans model to X_scaled and create the cluster labels
kmeans = KMeans(n_clusters=10, random_state=0)
X["Cluster"] = kmeans.fit_predict(X_scaled)

In [ ]:
#Code by Ryan H. https://www.kaggle.com/ryanholbrook/clustering-with-k-means

#The only change here is Transported. In the original it was SalePrice

Xy = X.copy()
Xy["Cluster"] = Xy.Cluster.astype("category")
Xy["Transported"] = y
sns.relplot(
    x="value", y="Transported", hue="Cluster", col="variable",
    height=4, aspect=1, facet_kws={'sharex': False}, col_wrap=3,
    data=Xy.melt(
        value_vars=features, id_vars=["Transported", "Cluster"],
    ),
);

In [ ]:
score_dataset(X, y)

#ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [ ]:
#Code by Ryan H. https://www.kaggle.com/ryanholbrook/clustering-with-k-means

kmeans = KMeans(n_clusters=10, n_init=10, random_state=0)


# Create the cluster-distance features using `fit_transform`
X_cd = kmeans.fit_transform(X_scaled)


# Label features and join to dataset
X_cd = pd.DataFrame(X_cd, columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])])
X = X.join(X_cd)

In [ ]:
score_dataset(X, y)

#The same Error

ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [ ]:
_ = sns.regplot(df['Age'], df['Transported'])

#Acknowledgement:

Mutual Information by Ryan Holbrook
All script by Ryan Holbrook https://www.kaggle.com/mpwolke/exercise-mutual-information/edit

Feature Engineering Kaggle Micro-course by Ryan Holbrook.

https://www.kaggle.com/learn/feature-engineering

Ryan Holbrook https://www.kaggle.com/ryanholbrook/clustering-with-k-means

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Be patient. Marília Prata, @mpwolke Was here' )